In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from scipy.stats import multivariate_normal as norm
import pandas as pd
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
file_data_b_csv=r'/home/jacobo15defrutos/AVS9/5-Pruebas/total_imgs.csv'
df= pd.read_csv(file_data_b_csv)
df_train, df_test = train_test_split(df, test_size=0.2,random_state=42)
print("Train shape: ",df_train.shape)
print("Test shape: ",df_test.shape)

Train shape:  (268, 2)
Test shape:  (67, 2)


In [3]:
labels_train= df_train['Label']
train_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_train['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img=img.ravel()
    train_list.append(img)
train= np.array(train_list)
train.shape

(268, 50176)

In [ ]:
lda=LDA(n_components=2)
lda.fit(train,labels_train)

In [12]:
lda=PCA(n_components=200)
lda.fit(train)

PCA(n_components=200)

In [13]:
train_SANO_list=[]
train_CMV_list=[]
train_SURV_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img=img.ravel()
    if 'SANO' in name:
        train_SANO_list.append(img)
    elif 'CMV' in name:
        train_CMV_list.append(img)
    elif 'SURV' in name:
        train_SURV_list.append(img)
train_SANO= np.array(train_SANO_list)
train_CMV= np.array(train_CMV_list)
train_SURV= np.array(train_SURV_list)
train_SURV.shape

(92, 50176)

In [14]:
labels_test= df_test['Label']
test_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_test['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img_dense = img.ravel()
    test_list.append(img_dense)
test= np.array(test_list)
test.shape

(67, 50176)

In [15]:
test_SANO_list=[]
test_CMV_list=[]
test_SURV_list=[]
target_size = (224, 224)
folder= '/home/jacobo15defrutos/AVS9/5-Pruebas/Total_images/'
for name in df_test['Name']:
    path= folder + name
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img,target_size)
    img=img.ravel()
    if 'SANO' in name:
        test_SANO_list.append(img)
    elif 'CMV' in name:
        test_CMV_list.append(img)
    elif 'SURV' in name:
        test_SURV_list.append(img)
test_SANO= np.array(test_SANO_list)
test_CMV= np.array(test_CMV_list)
test_SURV= np.array(test_SURV_list)
label_test_sano=0*np.ones(len(test_SANO))
label_test_cmv=1*np.ones(len(test_CMV))
label_test_surv=2*np.ones(len(test_SURV))

In [16]:
x_train=lda.transform(train)
x_test=lda.transform(test)
neigh = KNeighborsClassifier(n_neighbors=3,weights='distance',p=2)
neigh.fit(train,labels_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [17]:
pred=neigh.predict(test)
#pred= np.argmax(np.c_[p_sano,p_cmv,p_surv],axis=1)
pred

array([0, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 2, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0,
       2, 1, 1, 0, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0,
       1])

In [18]:
from sklearn.metrics import accuracy_score
acc_total= accuracy_score(labels_test,pred)
print("Total Accuracy: ",acc_total*100,"%")

Total Accuracy:  44.776119402985074 %


### Calculate each class accuracy

In [19]:
acc_sano = np.sum(pred[labels_test == 0] == 0)/len(label_test_sano) * 100
acc_cmv = np.sum(pred[labels_test == 1] == 1)/len(label_test_cmv) * 100
acc_surv = np.sum(pred[labels_test == 2] == 2)/len(label_test_surv) * 100
print("Class SANO Accuracy: {:.2f} %".format(acc_sano))
print("Class CMV Accuracy: {:.2f} %".format(acc_cmv))
print("Class SURV Accuracy: {:.2f} %".format(acc_surv))

Class SANO Accuracy: 58.33 %
Class CMV Accuracy: 23.08 %
Class SURV Accuracy: 58.82 %
